In [48]:
!pip install torch

You should consider upgrading via the '/Users/nikasakandelidze/deeplearning/deeplearning/bin/python -m pip install --upgrade pip' command.


In [49]:
import torch

In [47]:
names = open('names.txt').read().split("\n")

In [3]:
print(len(names))

32033


In [5]:
print(min([len(x) for x in names]))
print((max([len(x) for x in names])))

2
15


In [97]:
# bi-gram model
stats = torch.zeros([28, 28], dtype=torch.int32)

def get_indexes(prev, nxt):
    zero_based_index =  lambda i : ord(i) - 97
    if prev == '<START>':
        return 0, zero_based_index(nxt)
    elif nxt == '<END>':
        return zero_based_index(prev), 27
    else:
        return zero_based_index(prev), zero_based_index(nxt)
    

for name in names:
    word = ['<START>'] + list(name) + ['<END>']
    for prev, nxt in zip(word, word[1:]):
        i,j = get_indexes(prev, nxt)
        stats[i][j] += 1

max_idx = stats.argmax()
y_max, x_max = max_idx // 28, max_idx % 28

print(chr(y_max+97), chr(x_max+97), stats[y_max][x_max])


stats[stats == 0] = stats[y_max][x_max] + 10 # custom solution to not interfere with min calc
min_idx = stats.argmin()
y_min, x_min = min_idx // 28, min_idx % 28

print(chr(y_min+97), chr(x_min+97), stats[y_min][x_min])

n | tensor(6763, dtype=torch.int32)
b c tensor(1, dtype=torch.int32)
